## 해당 코드는 step3의 마지막 단계로 비중을 조절해주는 코드입니다.
### step2인 국면 모델에서 나온 자산군별 비중값을 읽어와야합니다.
### 과정을 보여드리기 위해 class화를 하지는 않았습니다.

In [249]:
import pandas as pd
import datetime
import numpy as np
from typing import List
from scipy.optimize import minimize

In [253]:
dir='C:/Users/User/ficc_quant/TICC/'

weight_pd=pd.read_excel(dir+'weight_pd_risk.xlsx',index_col=0)

#### 기간에 따른 자산군별 비중 값입니다.

In [254]:
weight_pd

,dollar,highyield,TIPS,US_bond,gold
2013-12-31,2.286085e-01,0.271232,0.166875,0.236484,0.096801
2014-01-31,2.289731e-01,0.270757,0.166697,0.236825,0.096748
2014-02-28,2.297806e-01,0.269678,0.166786,0.237108,0.096648
2014-03-31,2.321367e-01,0.266688,0.167488,0.238231,0.095456
2014-04-30,2.363313e-01,0.239940,0.175582,0.249998,0.098150
...,...,...,...,...,...
2022-07-31,-4.293690e-15,0.197346,0.269055,0.401699,0.131899
2022-08-31,-1.941090e-15,0.198347,0.269255,0.400428,0.131970
2022-09-30,-1.737507e-15,0.200393,0.262970,0.401365,0.135272
2022-10-31,-1.531039e-15,0.200892,0.264762,0.398502,0.135843


#### 미리 자산군별 ETF 티커를 리스트로 저장해둡니다.

In [255]:
corporate_ETF_list=['LQD','VCLT','VCIT','VCSH','GSY']
highyield_ETF_list=['HYG','SHYG']
T_bond_ETF_list=['GOVT','TLT','IEF','SHY','SHV']
gold_ETF_list=['IAU']
TIPS_ETF_list=['LTPZ','VTIP']



#### 다음은 ETF 데이터를 모두 불러옵니다. ficc_quant 폴더 중 ETF_data 폴더에 모두 저장해두었습니다. 데이터 출처는 모두 블룸버그 터미널입니다.

In [256]:
import os
path = 'C:/Users/User/ficc_quant/ETF_data'

file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')] 

In [257]:
file_list_py
name_list=[]
for index,file in enumerate(file_list_py):
    
    file_data=pd.read_excel(path+'/'+file)
    #print(file_data)
    name=file_data.iloc[0].index[1]
    file_data=pd.DataFrame(file_data.iloc[6:,:2]).reset_index(drop=True).iloc[::-1]
    file_data.columns=['Date',name]
    name_list.append(name)
    if index>0:
        all_file_data=pd.merge(all_file_data,file_data, how='outer')
    else:
        all_file_data=file_data
    
all_file_data.Date=pd.to_datetime(all_file_data.Date,errors='coerce')
all_file_data=all_file_data.loc[all_file_data.Date != None].reset_index(drop=True)
print(all_file_data)
all_file_data=all_file_data.sort_values(by=['Date'])
all_file_data=all_file_data.ffill()
ETF_data=all_file_data

           Date  \
0    1990-04-11   
1    1990-04-12   
2    1990-04-17   
3    1990-04-18   
4    1990-04-19   
...         ...   
8566 2023-01-02   
8567 2015-12-25   
8568 2016-12-26   
8569 2017-12-25   
8570 2020-12-25   

     XAU BGN Curncy                                                    \
0                                                 374.5                 
1                                                 375.5                 
2                                                   374                 
3                                                374.75                 
4                                                375.75                 
...                                                 ...                 
8566                                                NaN                 
8567                                                NaN                 
8568                                                NaN                 
8569                                     

#### ETF 데이터를 모두 하나의 데이터프레임으로 만든 뒤 칼럼을 붙혀줍니다.

In [258]:
ETF_data=ETF_data.set_index('Date')
ETF_columns=[i.replace('.xlsx','')for i in file_list_py]
ETF_data.columns=ETF_columns

In [259]:
ETF_data

,gold,GOVT,GSY,HSV,HYG,IEF,LQD,LTPZ,SHV,SHY,SHYG,TIPS,TLT,VCIT,VCLT,VCSH,VTIP
Date,,,,,,,,,,,,,,,,,
1990-04-11,374.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-12,375.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-17,374.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-18,374.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-19,375.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09,1871.81,104.96,49.57,128.20,75.80,98.63,109.03,61.51,110.00,268.08,179.2979,330.6085,102.61,79.51,79.74,75.81,97.24
2023-01-10,1877.03,104.48,49.57,128.20,75.71,98.01,108.66,60.69,109.99,267.92,179.1167,329.2170,100.97,79.35,79.28,75.77,97.18
2023-01-11,1875.69,104.98,49.58,128.22,76.21,98.64,109.59,61.43,110.02,268.09,179.6783,330.8064,102.58,79.80,80.35,75.92,97.26


#### 이후 데이터가 비어있을경우는 전날짜로 채워준 뒤 변화율을 구해줍니다.

In [260]:
ETF_data=ETF_data.ffill().dropna()
ETF_data=ETF_data.pct_change().iloc[1:]

In [261]:
ETF_data

,gold,GOVT,GSY,HSV,HYG,IEF,LQD,LTPZ,SHV,SHY,SHYG,TIPS,TLT,VCIT,VCLT,VCSH,VTIP
Date,,,,,,,,,,,,,,,,,
2010-12-06,0.006838,0.004114,0.000000,0.000084,-0.000581,0.006257,0.006979,0.009781,0.000091,0.001141,0.000835,0.005365,0.010866,0.003125,0.009454,0.002317,0.002403
2010-12-07,-0.015220,-0.010471,-0.000804,0.000000,0.005170,-0.016580,-0.009302,-0.015784,0.000000,-0.002239,0.001848,-0.011376,-0.027978,-0.008478,-0.014625,-0.004197,-0.001698
2010-12-08,-0.014300,-0.003451,0.000000,0.000000,-0.003230,-0.008535,-0.006444,-0.017311,0.000000,-0.001142,-0.000897,-0.007735,-0.003636,-0.008613,-0.001979,-0.002358,-0.003400
2010-12-09,0.003676,-0.000115,0.000000,0.000000,-0.002121,0.002125,0.002410,-0.001487,0.000000,-0.000286,0.000730,-0.001792,0.006994,0.001333,0.003940,-0.001421,-0.000202
2010-12-10,-0.000707,-0.002193,0.000604,0.000000,0.000448,-0.008485,-0.004622,-0.005572,-0.000181,-0.000082,0.000125,-0.002291,-0.003775,-0.004435,-0.002287,-0.002069,-0.000903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-09,0.003280,0.002483,0.001010,0.000078,0.003575,0.002541,0.003959,0.008526,0.000091,0.001083,0.004319,0.002728,0.005389,0.003787,0.004409,0.001189,0.000926
2023-01-10,0.002789,-0.004573,0.000000,0.000000,-0.001187,-0.006286,-0.003394,-0.013331,-0.000091,-0.000597,-0.001011,-0.004209,-0.015983,-0.002012,-0.005769,-0.000528,-0.000617
2023-01-11,-0.000714,0.004786,0.000202,0.000156,0.006604,0.006428,0.008559,0.012193,0.000273,0.000635,0.003135,0.004828,0.015945,0.005671,0.013496,0.001980,0.000823


In [262]:
ETF_data.to_excel('all_ETF_data.xlsx')

#### 다음은 리스크 패리티에 필요한 함수들입니다. 해당 알고리즘은 아래의 깃허브에서 가져온 코드입니다.
#### 출처 : https://github.com/develly/risk_parity_optimizer/blob/main/risk_parity_strategy.py

In [263]:


TOLERANCE = 1e-20


def get_weights(num_of_asset: int):
    # Equal weight example.
    weight = 1 / num_of_asset
    weights = np.repeat(weight, num_of_asset)
    return weights


def get_covariances(data):
    # Calculate covariance matrix.
    covariance = data.cov()
    return covariance


def _get_risk_contribution(weights, covariance):
    # Optional: convert dataframe to numpy array.
    covariance = covariance.to_numpy()
    # Convert 1d array to 2d array (n x 1).
    weights = weights.reshape(-1, 1)
    # Calculate portfolio variance.
    variance = weights.T @ covariance @ weights
    # Calculate portfolio sigma.
    sigma = np.sqrt(variance)
    # Calculate mrc.
    mrc = 1 / sigma * (covariance @ weights)
    # Calculate rc.
    rc = np.multiply(weights, mrc)
    # Normalize.
    rc = rc / sum(rc)
    return rc


def _get_error(weights, covariance):
    # Get rc.
    rc_matrix = _get_risk_contribution(weights, covariance)
    # Calculate error.
    error = np.sum(np.square(rc_matrix - rc_matrix.T))
    return error


def _get_opt_weights(covariance: List[float]):
    # Constraints: sum of weights is 1, weight is more than 0.
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}, {'type': 'ineq', 'fun': lambda x: x})
    # Setting max iteration number.
    options = {'maxiter': 800}
    # Get initial weights (Equal weights).
    initial_weights = get_weights(len(covariance))
    # Optimisation process in scipy.
    optimize_result = minimize(fun=_get_error,
                               x0=initial_weights,
                               args=covariance,
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options=options)
    # Recover the weights from the optimised object.
    weights = optimize_result.x
    # It returns the optimised weights.
    return weights


#### 이제부터는 각 자산군별 ETF들을 불러와 각각 자산군마다 따로 리스크패리티 전략을 취해주는 과정입니다.

#### 날짜마다 비중을 가져와 자산군 비중마다 if문에서 리스크 패리티로 최적화를 시켜줍니다.

#### 회사채와 하이일드 채권은 성격이 비슷하여 하이일드 채권의 비중을 2등분해서 각각 따로 리스크패리티를 진행해주었습니다.


In [264]:

ticker_list=[]
start_day_list=[]
last_weight_pd=pd.DataFrame()
for index,rebalance_day in enumerate(weight_pd.index):
    ticker_list=[]
    ETF_weight_list=[]
    weight_list=weight_pd.loc[rebalance_day,:]
    for asset,weight in zip(weight_list.index,weight_list):
        print(asset,weight)
        if asset=='dollar': #달러일경우
            ETF_weight_list.append(weight)
            ticker_list.append('dollar')
            continue
        
        elif asset=='highyield': # 하이일드 자산군일경우 
            weight=weight/2
            tmt_profit_pd_highyield=ETF_data.loc[:rebalance_day,highyield_ETF_list]
            
            # Get covariance.
            covariance_matrix = get_covariances(tmt_profit_pd_highyield)
            # Get optimum weights.
            weight_matrix = _get_opt_weights(covariance_matrix)
            weight_matrix = list(np.array(weight_matrix)*weight)
            #비중을 ETF_wright_list에 넣어줍니다.
            ETF_weight_list.extend(weight_matrix)
            #ETF 티커도 넣어줍니다.
            ticker_list.extend(highyield_ETF_list)
            
            
            
            tmt_profit_pd_corporate=ETF_data.loc[:rebalance_day,corporate_ETF_list]
            
            # Get covariance.
            covariance_matrix = get_covariances(tmt_profit_pd_corporate)
            # Get optimum weights.
            weight_matrix = _get_opt_weights(covariance_matrix)
            weight_matrix = list(np.array(weight_matrix)*weight)
            ETF_weight_list.extend(weight_matrix)
            ticker_list.extend(corporate_ETF_list)
            
        elif asset=='TIPS': #TIPS 자산군일경우
            tmt_profit_pd_TIPS=ETF_data.loc[:rebalance_day,TIPS_ETF_list]
            
            # Get covariance.
            covariance_matrix = get_covariances(tmt_profit_pd_TIPS)
            # Get optimum weights.
            weight_matrix = _get_opt_weights(covariance_matrix)
            weight_matrix = list(np.array(weight_matrix)*weight)
            print(weight_matrix)
            
            ETF_weight_list.extend(weight_matrix)
            ticker_list.extend(TIPS_ETF_list)
            
            
        elif asset=='US_bond': #국채일경우
            tmt_profit_pd_T_bond=ETF_data.loc[:rebalance_day,T_bond_ETF_list]
            
            # Get covariance.
            covariance_matrix = get_covariances(tmt_profit_pd_T_bond)
            # Get optimum weights.
            weight_matrix = _get_opt_weights(covariance_matrix)
            weight_matrix = list(np.array(weight_matrix)*weight)
            print(weight_matrix)
            ETF_weight_list.extend(weight_matrix)
            
            ticker_list.extend(T_bond_ETF_list)
            
        elif asset=='gold':
            ETF_weight_list.append(weight)
            ticker_list.append('gold')
        
    
        
    last_weight_data=pd.DataFrame(ETF_weight_list).T
    last_weight_data.columns=ticker_list
    # start_day는 매수날짜라고 생각하시면 되겠습니다. 모두 월초입니다.
    start_day=rebalance_day+datetime.timedelta(1)
    
    start_day_list.append(start_day)
    
    if index==0:
        last_weight_pd=pd.concat([last_weight_pd,last_weight_data])
    else:
        last_weight_pd=last_weight_pd.append(last_weight_data)
    
    #최종 데이터프레임은 last_weight_pd입니다.
    print(last_weight_pd)
    
            
            
    
    

dollar 0.2286085306542467
highyield 0.2712317215380677
TIPS 0.1668751594996284
[0.022008784809106622, 0.14486637469052177]
US_bond 0.23648377829702
[0.004459425551566121, 0.0012184391538845058, 0.0027379659189504845, 0.032249734165661174, 0.19581821350695772]
gold 0.09680081001103716
     dollar       HYG      SHYG       LQD     VCLT      VCIT      VCSH  \
0  0.228609  0.033534  0.102082  0.006749  0.00417  0.007666  0.023005   

        GSY      LTPZ      VTIP      GOVT       TLT       IEF      SHY  \
0  0.094026  0.022009  0.144866  0.004459  0.001218  0.002738  0.03225   

        SHV      gold  
0  0.195818  0.096801  
dollar 0.2289731261443009
highyield 0.2707572164143606
TIPS 0.1666968583161766
[0.021944820702707292, 0.1447520376134693]
US_bond 0.2368249150066831
[0.004451606542961827, 0.0012189094481093064, 0.0027270707990346608, 0.03194272400940649, 0.1964846042071708]
gold 0.0967478841184787
     dollar       HYG      SHYG       LQD      VCLT      VCIT      VCSH  \
0  0.228609

C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)


TIPS 0.1667858631984094
[0.02191246763810939, 0.14487339556030002]
US_bond 0.2371077893354797
[0.004502066596299002, 0.0012313103186486758, 0.0027562996210534997, 0.03214394179915187, 0.19647417100032666]
gold 0.09664809551575315
     dollar       HYG      SHYG       LQD      VCLT      VCIT      VCSH  \
0  0.228609  0.033534  0.102082  0.006749  0.004170  0.007666  0.023005   
0  0.228973  0.033473  0.101905  0.006803  0.004196  0.007716  0.023054   
0  0.229781  0.033313  0.101525  0.006774  0.004174  0.007698  0.023026   

        GSY      LTPZ      VTIP      GOVT       TLT       IEF       SHY  \
0  0.094026  0.022009  0.144866  0.004459  0.001218  0.002738  0.032250   
0  0.093610  0.021945  0.144752  0.004452  0.001219  0.002727  0.031943   
0  0.093168  0.021912  0.144873  0.004502  0.001231  0.002756  0.032144   

        SHV      gold  
0  0.195818  0.096801  
0  0.196485  0.096748  
0  0.196474  0.096648  
dollar 0.2321366678688309
highyield 0.2666875547233567
TIPS 0.1674884615

C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)
C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)


     dollar       HYG      SHYG       LQD      VCLT      VCIT      VCSH  \
0  0.228609  0.033534  0.102082  0.006749  0.004170  0.007666  0.023005   
0  0.228973  0.033473  0.101905  0.006803  0.004196  0.007716  0.023054   
0  0.229781  0.033313  0.101525  0.006774  0.004174  0.007698  0.023026   
0  0.232137  0.032913  0.100431  0.006688  0.004143  0.007594  0.022816   

        GSY      LTPZ      VTIP      GOVT       TLT       IEF       SHY  \
0  0.094026  0.022009  0.144866  0.004459  0.001218  0.002738  0.032250   
0  0.093610  0.021945  0.144752  0.004452  0.001219  0.002727  0.031943   
0  0.093168  0.021912  0.144873  0.004502  0.001231  0.002756  0.032144   
0  0.092103  0.022036  0.145453  0.004513  0.001236  0.002752  0.031940   

        SHV      gold  
0  0.195818  0.096801  
0  0.196485  0.096748  
0  0.196474  0.096648  
0  0.197790  0.095456  
dollar 0.2363313080983651
highyield 0.2399396855320027
TIPS 0.1755816200816684
[0.02304996994438606, 0.15253165013728234]
US_bon

...

C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)


[0.03492657017588278, 0.22804341145173065]
US_bond 0.4013645845349978
[0.010130103122948956, 0.0032839096158347697, 0.007179206967575183, 0.04854058729362992, 0.3322307775350089]
gold 0.1352721875297438
          dollar       HYG      SHYG       LQD      VCLT      VCIT      VCSH  \
0   2.286085e-01  0.033534  0.102082  0.006749  0.004170  0.007666  0.023005   
0   2.289731e-01  0.033473  0.101905  0.006803  0.004196  0.007716  0.023054   
0   2.297806e-01  0.033313  0.101525  0.006774  0.004174  0.007698  0.023026   
0   2.321367e-01  0.032913  0.100431  0.006688  0.004143  0.007594  0.022816   
0   2.363313e-01  0.029590  0.090380  0.006017  0.003727  0.006830  0.020516   
..           ...       ...       ...       ...       ...       ...       ...   
0  -4.361744e-16  0.031339  0.065797  0.006702  0.004595  0.009577  0.017885   
0  -6.987032e-16  0.031474  0.065495  0.006653  0.004570  0.009458  0.017730   
0  -4.293690e-15  0.032037  0.066636  0.006748  0.004645  0.009566  0.017974 

C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)
C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)


[0.035734623814820156, 0.23133422108697213]
US_bond 0.392492223486041
[0.009898763645473957, 0.0032319752298400803, 0.007010463483689015, 0.046613491107777266, 0.3257375300192589]
gold 0.1360359513482528
          dollar       HYG      SHYG       LQD      VCLT      VCIT      VCSH  \
0   2.286085e-01  0.033534  0.102082  0.006749  0.004170  0.007666  0.023005   
0   2.289731e-01  0.033473  0.101905  0.006803  0.004196  0.007716  0.023054   
0   2.297806e-01  0.033313  0.101525  0.006774  0.004174  0.007698  0.023026   
0   2.321367e-01  0.032913  0.100431  0.006688  0.004143  0.007594  0.022816   
0   2.363313e-01  0.029590  0.090380  0.006017  0.003727  0.006830  0.020516   
..           ...       ...       ...       ...       ...       ...       ...   
0  -4.293690e-15  0.032037  0.066636  0.006748  0.004645  0.009566  0.017974   
0  -1.941090e-15  0.032179  0.066994  0.006726  0.004643  0.009518  0.017960   
0  -1.737507e-15  0.032559  0.067638  0.006729  0.004651  0.009493  0.017999

C:\Users\User\AppData\Local\Temp\ipykernel_2784\4068784161.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  last_weight_pd=last_weight_pd.append(last_weight_data)


In [265]:

last_weight_pd.index=start_day_list

In [266]:
last_weight_pd

,dollar,HYG,SHYG,LQD,VCLT,VCIT,VCSH,GSY,LTPZ,VTIP,GOVT,TLT,IEF,SHY,SHV,gold
2014-01-01,2.286085e-01,0.033534,0.102082,0.006749,0.004170,0.007666,0.023005,0.094026,0.022009,0.144866,0.004459,0.001218,0.002738,0.032250,0.195818,0.096801
2014-02-01,2.289731e-01,0.033473,0.101905,0.006803,0.004196,0.007716,0.023054,0.093610,0.021945,0.144752,0.004452,0.001219,0.002727,0.031943,0.196485,0.096748
2014-03-01,2.297806e-01,0.033313,0.101525,0.006774,0.004174,0.007698,0.023026,0.093168,0.021912,0.144873,0.004502,0.001231,0.002756,0.032144,0.196474,0.096648
2014-04-01,2.321367e-01,0.032913,0.100431,0.006688,0.004143,0.007594,0.022816,0.092103,0.022036,0.145453,0.004513,0.001236,0.002752,0.031940,0.197790,0.095456
2014-05-01,2.363313e-01,0.029590,0.090380,0.006017,0.003727,0.006830,0.020516,0.082879,0.023050,0.152532,0.004740,0.001299,0.002888,0.033475,0.207596,0.098150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,-4.293690e-15,0.032037,0.066636,0.006748,0.004645,0.009566,0.017974,0.059740,0.035322,0.233734,0.010236,0.003284,0.007268,0.049876,0.331035,0.131899
2022-09-01,-1.941090e-15,0.032179,0.066994,0.006726,0.004643,0.009518,0.017960,0.060326,0.035471,0.233785,0.010160,0.003274,0.007216,0.049153,0.330625,0.131970
2022-10-01,-1.737507e-15,0.032559,0.067638,0.006729,0.004651,0.009493,0.017999,0.061324,0.034927,0.228043,0.010130,0.003284,0.007179,0.048541,0.332231,0.135272
2022-11-01,-1.531039e-15,0.032624,0.067822,0.006701,0.004634,0.009431,0.017943,0.061737,0.035362,0.229400,0.010036,0.003265,0.007107,0.047718,0.330376,0.135843


In [267]:
last_weight_pd.to_excel('last_portfolio_weight_risk.xlsx')